importar dados

In [58]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

In [59]:
biog = pd.read_csv("biodegradable_a.csv", sep=",")

X = biog.values[:, 0:41]
y = biog["Biodegradable"].values

y[y == "RB"] = 1.0
y[y == "NRB"] = 0.0

X_train_IVS, X_IVS, y_train_IVS, y_IVS = train_test_split(X, y.astype(float), test_size=0.33, random_state=0)
X_train, X_test, y_train, y_test = train_test_split(X_train_IVS, y_train_IVS, test_size=0.33, random_state=0)

X_cols = biog.columns[0:41]

countRB = 0
countNRB = 0
for i in y:
    if(i == 1): countRB += 1
    elif(i == 0): countNRB += 1

print(countRB, countNRB)

# pd.DataFrame(X, columns=X_cols)

3825 739


Check missing values

In [60]:
# check number of missing values in each column.

# dataset without lines with NaN values
biog_nan = biog.dropna()
#print(len(biog_nan))

print("No. of rows original dataset = {}".format(len(biog)))
print("No. of rows without NaN = {} ({}%)".format(len(biog_nan), round(len(biog_nan) / len(biog) * 100)))


# check number of missing values in each column.
missValues = biog.isna().sum()

missValues = missValues.to_frame()
print("Threshold number of missing values per column = {} (15%)".format(str(round(0.15 * len(X)))))

# print(missValues)

No. of rows original dataset = 4564
No. of rows without NaN = 889 (19%)
Threshold number of missing values per column = 685 (15%)


Escalar os dados (Standard Scaler)

In [62]:
import statistics

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

# df2 = pd.DataFrame(XscaledStS, columns=X_cols)

# for i in range(len(XscaledStS[0])):
#     values = XscaledStS[:, i]
#     plt.hist(values)
#     plt.title(X_cols[i] + " " + str(i))
#     plt.show()
#     plt.close()

Metodo de Imputação

In [63]:
useMean = [0, 1, 7, 11, 12, 13, 14, 21, 26, 27, 38]
useMedian = np.setdiff1d(range(41), useMean)

imputar dados

In [64]:
# Impute missing values
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer 

# strategy = mean or median (for numerical data)
imputerMean = SimpleImputer(missing_values = np.nan, strategy = "mean") 
imputerMedian = SimpleImputer(missing_values = np.nan, strategy = "median") 

ct = ColumnTransformer(
    [('mean_imputer', imputerMean, useMean),
     ('median_imputer', imputerMedian, useMedian)]
)

ct.fit(X_train_scaled)

X_train_processed = ct.transform(X_train)
X_test_processed = ct.transform(X_test)

# Feature selection

Correlation

In [76]:
N, M = X_train.shape

v = np.hstack((y_train.reshape((N, 1)), X_train_processed))

pd.DataFrame(v)
corrMatrix = pd.DataFrame((np.corrcoef(v.T.astype(float))), columns=X_cols.insert(0, "Y")).iloc[0].sort_values()[0:10]

selectedCorr = list(corrMatrix.index)

print(corrMatrix)
print("The 10 selected features are:", selectedCorr)

SdssC      -0.493378
nHDon      -0.459505
nN_N       -0.447186
F02_CN     -0.426871
SM6_B      -0.418151
LOC        -0.411746
F03        -0.391805
Psi_i_1d   -0.381357
F03_CO     -0.354265
SpMax_L    -0.344159
Name: 0, dtype: float64
The 10 selected features are: ['SdssC', 'nHDon', 'nN_N', 'F02_CN', 'SM6_B', 'LOC', 'F03', 'Psi_i_1d', 'F03_CO', 'SpMax_L']


Random Forest Feature Selection

In [66]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel


sel = SelectFromModel(RandomForestClassifier(), max_features=10)
sel.fit(X_train_processed, y_train)

selectedForest = []
selectedForestIndixes = []

for feature_list_index in sel.get_support(indices=True):
    selectedForestIndixes.append(feature_list_index)
    selectedForest.append(X_cols[feature_list_index])
    print("Feature: ", X_cols[feature_list_index], "| Importance: ", sel.estimator_.feature_importances_[feature_list_index])

Feature:  SpMax_L | Importance:  0.059076657184318024
Feature:  C | Importance:  0.038278926617807
Feature:  F03 | Importance:  0.03354248980134454
Feature:  SdssC | Importance:  0.11611419286831129
Feature:  LOC | Importance:  0.06674559259910746
Feature:  SM6_L | Importance:  0.05859079684160149
Feature:  F03_CO | Importance:  0.05243329170263543
Feature:  nN_N | Importance:  0.06310685782709276
Feature:  nHDon | Importance:  0.058294807925505676
Feature:  nX | Importance:  0.032726752350138714


StepWise (Forward)

In [68]:
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.tree import DecisionTreeClassifier

N,M = X_train_processed.shape

dtc = DecisionTreeClassifier()
sfs = SequentialFeatureSelector(dtc, n_features_to_select=10, direction="forward")

sfs.fit(X_train_processed, y_train)

features = sfs.get_support()
ST_for_selected_features = np.arange(M)[features]

print(X_cols[ST_for_selected_features])

Index(['SdssC', 'HyWi_B', 'LOC', 'SM6_L', 'F03_CO', 'nArNO2', 'nCIR',
       'Psi_i_1d', 'C_026', 'nHDon'],
      dtype='object')


StepWise (BackWard)

In [69]:
dtc = DecisionTreeClassifier()
sfs = SequentialFeatureSelector(dtc, n_features_to_select=10, direction="backward")

sfs.fit(X_train_processed, y_train)

features = sfs.get_support()
ST_back_selected_features = np.arange(M)[features]

print(X_cols[ST_back_selected_features])

Index(['F01', 'C', 'F03', 'SdssC', 'SM6_L', 'F03_CO', 'SpPosA_B', 'C_026',
       'nHDon', 'nN'],
      dtype='object')


Indexes of Features Selcted

In [96]:
corr_index = []
rand_forest_index = selectedForestIndixes
step_for_index = list(ST_for_selected_features)
step_back_index = list(ST_back_selected_features)

count = 0
for col in X_cols:
    if col in selectedCorr:
        corr_index.append(count)
    count += 1

print(corr_index)
print(rand_forest_index)
print(step_for_index)
print(step_back_index)

[0, 10, 11, 13, 15, 18, 27, 33, 34, 38]
[0, 7, 10, 11, 13, 14, 15, 18, 34, 40]
[11, 12, 13, 14, 15, 19, 22, 27, 32, 34]
[3, 7, 10, 11, 14, 15, 21, 32, 34, 37]


In [98]:
selec_feat_indexes = [corr_index,
                      rand_forest_index,
                      step_for_index,
                      step_back_index]

selec_feat_names = ["Correlation",
                      "Random Forest",
                      "StepWise For",
                      "StepWise Back"]

Define classification metrics

In [100]:
from sklearn.metrics import f1_score, matthews_corrcoef, confusion_matrix, accuracy_score

def printClassResults(truth, preds):
    print("The F1 score is: %7.4f" % f1_score(truth, preds))
    print("The Matthews correlation coefficient is: %7.4f" % matthews_corrcoef(truth, preds))
    print()
    print("This is the Confusion Matrix")
    print(pd.DataFrame(confusion_matrix(truth, preds)))
    print("\n")

# printClassResults(y, preds)

# First Model

KNN

In [111]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import sys


def gaussian(dsts):
    kernel_width = .5
    weights = np.exp(-(dsts**2)/kernel_width)
    return weights


for i in range(4):
    print("Feature selection method: " + selec_feat_names[i])

    X_train = X_train_processed[:, selec_feat_indexes[i]]
    X_test = X_test_processed[:, selec_feat_indexes[i]]

    f1gauss = []
    f1distance = []
    f1noWei = []

    bestMCC = sys.float_info.min
    bestK = 0
    bestWeight = ""

    for k in range(2, 20, 2):
        knn = KNeighborsClassifier(n_neighbors=k, weights=gaussian)
        knn.fit(X_train, y_train)
        preds = knn.predict(X_test)
        f1gauss.append(f1_score(y_test, preds))
        print("Nº Neighbours " + str(k) + " | weights=gaussian" + "\n")
        printClassResults(y_test, preds)

        knn1 = KNeighborsClassifier(n_neighbors=k, weights="distance")
        knn1.fit(X_train, y_train)
        preds = knn1.predict(X_test)
        f1distance.append(f1_score(y_test, preds))
        print("Nº Neighbours " + str(k) + " | weights=distance" + "\n")
        printClassResults(y_test, preds)

        knn2 = KNeighborsClassifier(n_neighbors=k)
        knn2.fit(X_train, y_train)
        preds = knn2.predict(X_test)
        f1noWei.append(f1_score(y_test, preds))
        print("Nº Neighbours " + str(k) + " | no weights" + "\n")
        printClassResults(y_test, preds)

    # plt.plot(range(2, 20, 2), f1gauss, label="gaussian")
    # plt.plot(range(2, 20, 2), f1distance, label="distance")
    # plt.plot(range(2, 20, 2), f1noWei, label="no weights")
    # plt.legend()
    # plt.grid()
    # plt.show()

Feature selection method: Correlation
Nº Neighbours 2 | weights=gaussian

The F1 score is:  0.9637
The Matthews correlation coefficient is:  0.7643

This is the Confusion Matrix
     0    1
0  124   36
1   26  823


Nº Neighbours 2 | weights=distance

The F1 score is:  0.9637
The Matthews correlation coefficient is:  0.7643

This is the Confusion Matrix
     0    1
0  124   36
1   26  823


Nº Neighbours 2 | no weights

The F1 score is:  0.9548
The Matthews correlation coefficient is:  0.7299

This is the Confusion Matrix
     0    1
0  130   30
1   46  803


Nº Neighbours 4 | weights=gaussian

The F1 score is:  0.9656
The Matthews correlation coefficient is:  0.7719

This is the Confusion Matrix
     0    1
0  121   39
1   20  829


Nº Neighbours 4 | weights=distance

The F1 score is:  0.9633
The Matthews correlation coefficient is:  0.7571

This is the Confusion Matrix
     0    1
0  120   40
1   23  826


Nº Neighbours 4 | no weights

The F1 score is:  0.9613
The Matthews correlatio

Random Forest

In [113]:
X_train = X_train_processed[:, selec_feat_indexes[i]]
X_test = X_test_processed[:, selec_feat_indexes[i]]

for i in range(4):
        print("Feature selection method: " + selec_feat_names[i] + "\n")

        rf = RandomForestClassifier(n_estimators=50)
        rf.fit(X_train, y_train)

        preds = rf.predict(X_test)

        printClassResults(y_test, preds)

Feature selection method: Correlation

The F1 score is:  0.9706
The Matthews correlation coefficient is:  0.8134

This is the Confusion Matrix
     0    1
0  134   26
1   24  825


Feature selection method: Random Forest

The F1 score is:  0.9712
The Matthews correlation coefficient is:  0.8158

This is the Confusion Matrix
     0    1
0  133   27
1   22  827


Feature selection method: StepWise For

The F1 score is:  0.9713
The Matthews correlation coefficient is:  0.8142

This is the Confusion Matrix
     0    1
0  131   29
1   20  829


Feature selection method: StepWise Back

The F1 score is:  0.9730
The Matthews correlation coefficient is:  0.8267

This is the Confusion Matrix
     0    1
0  134   26
1   20  829




Bagging Classifier

In [128]:
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import BaggingClassifier, BaggingRegressor

X_train = X_train_processed[:, selec_feat_indexes[i]]
X_test = X_test_processed[:, selec_feat_indexes[i]]

for i in range(4):
    print("Feature selection method: " + selec_feat_names[i])
    f1Scores = []
    mccScores = []

    for n in range(2, 10, 2):
        print("Nº of estimators:" + " " + str(n) + "\n")
        bag = BaggingClassifier(base_estimator=GaussianNB(), n_estimators=n, random_state=22)
        bag.fit(X_train, y_train)
        preds = bag.predict(X_test)
        printClassResults(y_test, preds)
        f1Scores.append(f1_score(y_test, preds))
        mccScores.append(matthews_corrcoef(y_test, preds))
    
    # print(f1Scores)
    # plt.plot(range(2, 10, 2), f1Scores, label="F1")
    # plt.legend()
    # plt.grid()
    # plt.show()

    # print(mccScores)
    # plt.plot(range(2, 10, 2), mccScores, label="MCC")
    # plt.legend()
    # plt.grid()
    # plt.show()

Feature selection method: Correlation
Nº of estimators: 2

The F1 score is:  0.9567
The Matthews correlation coefficient is:  0.7173

This is the Confusion Matrix
     0    1
0  117   43
1   31  818


Nº of estimators: 4

The F1 score is:  0.9568
The Matthews correlation coefficient is:  0.7149

This is the Confusion Matrix
     0    1
0  115   45
1   29  820


Nº of estimators: 6

The F1 score is:  0.9568
The Matthews correlation coefficient is:  0.7149

This is the Confusion Matrix
     0    1
0  115   45
1   29  820


Nº of estimators: 8

The F1 score is:  0.9568
The Matthews correlation coefficient is:  0.7149

This is the Confusion Matrix
     0    1
0  115   45
1   29  820


Feature selection method: Random Forest
Nº of estimators: 2

The F1 score is:  0.9567
The Matthews correlation coefficient is:  0.7173

This is the Confusion Matrix
     0    1
0  117   43
1   31  818


Nº of estimators: 4

The F1 score is:  0.9568
The Matthews correlation coefficient is:  0.7149

This is the